In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/research'

In [2]:
os.chdir('../')
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8'

In [13]:
import kaggle
from dotenv import load_dotenv
import os
import subprocess
import cv2

load_dotenv()

True

In [8]:
competition_name = "data-science-bowl-2018"
dataset_filename = "stage1_train.zip"
root_dir = "/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/data"

In [4]:
!kaggle competitions download -c $competition_name -f $dataset_filename -p $root_dir

100%|█████████████████████████████████████▉| 79.0M/79.1M [00:04<00:00, 26.1MB/s]
100%|██████████████████████████████████████| 79.1M/79.1M [00:04<00:00, 18.0MB/s]


In [11]:
os.system("kaggle competitions download -c data-science-bowl-2018 -f stage1_train.zip -p '/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/data'")

500 - Internal Server Error - InternalServerError


256

In [13]:
info = os.system(f"kaggle competitions download -c {competition_name} -f {dataset_filename} -p {root_dir}")

 91%|█████████ | 72.0M/79.1M [00:03<00:00, 34.4MB/s]

100%|██████████| 79.1M/79.1M [00:03<00:00, 24.7MB/s]


In [14]:
info = subprocess.run(f"kaggle competitions download -c {competition_name} -f {dataset_filename} -p {root_dir}", shell=True)

 91%|█████████ | 72.0M/79.1M [00:03<00:00, 34.0MB/s]

100%|██████████| 79.1M/79.1M [00:03<00:00, 24.6MB/s]


In [15]:
print(info)

CompletedProcess(args='kaggle competitions download -c data-science-bowl-2018 -f stage1_train.zip -p /mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/data', returncode=0)


In [3]:
os.chdir('../')
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8'

In [4]:
path_folder = 'artifacts/data_ingestion/train'

In [5]:
folder_list = os.listdir(path_folder)

In [48]:
for dir in folder_list:
    img_pth = os.path.join(
        path_folder,
        dir,
        'images',
        dir + '.png'
    )
    
    img_ann = ''
    for ann in os.listdir(os.path.join(path_folder, dir, 'masks')):
        cell_line = '0'
        ann_img = cv2.imread(os.path.join(path_folder, dir, 'masks', ann), 0)
        contours, hierarchy = cv2.findContours(ann_img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        for dot in contours[0]:
            cell_line += ' ' + str(dot[0][1] / 255) + ' ' + str(dot[0][0] / 255)
        cell_line += '\n'
    
        img_ann += cell_line
    
    break

In [68]:
train_list = os.listdir('artifacts/data_transformation/validation')

In [69]:
len(train_list)

268

In [57]:
train_list = [s for s in train_list if '.png' in s]

In [61]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train_list, test_size=0.2, random_state=42,shuffle=True)

In [65]:
len(val)

134

In [3]:
from ultralytics import YOLO

In [5]:
model = YOLO('artifacts/model_trainer/yolov8n-seg.pt').to('cuda')

/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/.venv/lib/python3.10/site-packages/ultralytics/nn/tasks.py:336: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exp